In [350]:
import numpy as np
import pandas as pd
import os 
import glob
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pprint
import pygsheets

In [320]:
path = "/Users/egordon/Documents/Coding/budget/data/"
os.chdir(path)
files = glob.glob("*.csv") + glob.glob("*.CSV")

In [311]:
order = ['Source','Transaction Date', 'Description', 'Category',  'Amount',
        'Month','Type', 'Exclude', 'Only Exclusions', 'Without Exclusions']
def file_names(file):
    if file.find('Checking') == 0:
        return 'Checking'
    if file.find('Chase') == 0:
        return 'Chase'
    if file.find('Market') == 0:
        return 'Savings'
    if file.find('From') == 0:
        return 'Citi'
    if file.find('CreditCard')==0:
        return 'Wells Credit Card'
    else:
        return 'N/A'    

def checking_exclusions(description):
    if (description.find('CITI CARD')==0) \
        or (description.find('Acorns')==0) \
        or (description.find('CHASE CREDIT')==0) \
        or (description.find('ONLINE PAYMENT')==0):
        return 'x'
    elif (description.find('ONLINE TRANSFER')==0) \
        or (description.find('RECURRING TRANSFER')==0):
        return 'N'
    else:
        return ''

    
def compile_files( file , file_name ):
    
    if file_name == 'Chase':
        cur_data = pd.read_csv(file)
        cur_data.drop( columns = ['Post Date'],inplace= True )
        cur_data['Exclude'] = cur_data['Type'].replace({'Payment': 'x', 'Sale': ''})
        cur_data['Type'] = cur_data['Type'].replace({'Payment':'Income','Sale':'Spent'})
   
    if (file_name == 'Checking') or (file_name == 'Savings') or (file_name == 'Wells Credit Card'):
        cur_data = pd.read_csv(file, header =None)
        cur_data.drop(columns= [2,3], inplace=True)
        cur_data.columns = ['Transaction Date','Amount','Description']
        cur_data['Type'] = cur_data.apply(lambda x: 'Income' if x['Amount'] > 0 else 'Spent', axis =1 )
        
        cur_data['Exclude'] = cur_data['Description'].apply( lambda x: checking_exclusions(x) )
        
        #Need to Do some Pythong String Searching 
        cur_data['Category']  = ''
    
    if file_name == 'Citi':
        cur_data = pd.read_csv(file)
        cur_data.rename(columns = {'Date':'Transaction Date'}, inplace =True)
        cur_data['Amount'] = cur_data.apply( lambda x: -1*np.nansum( [x['Debit'], x['Credit'] ])  ,axis = 1 )
        cur_data['Type'] = cur_data.apply(lambda x: 'Income' if x['Amount'] > 0 else 'Spent', axis =1 )
        cur_data['Exclude'] = cur_data['Type'].replace({'Income': 'x', 'Spent': ''})
        cur_data['Category']  = ''
        cur_data.drop(columns= ['Debit','Credit'], inplace = True)
        
    cur_data['Month'] =  pd.DatetimeIndex(cur_data['Transaction Date']).month
    cur_data['Source'] = file_name
    cur_data['Only Exclusions'] =  cur_data.apply(lambda x: float(x['Amount']) if x['Exclude']== 'x' else 0, axis =1)
    cur_data['Without Exclusions'] =  cur_data.apply(lambda x: float(x['Amount']) if x['Exclude'] != 'x' else 0, axis =1)     
      
    return cur_data[order]

In [315]:
# Chase [3:4]
all_data = []
for file in files:  #files[1:4]: 
    print(file_names(file))
    all_data.append(compile_files( file, file_names(file)))
    
compiled_data = pd.concat(all_data)

Wells Credit Card
Checking
Savings
Chase
Citi


In [382]:
os.chdir("/Users/egordon/Documents/Coding/budget/")
gc = pygsheets.authorize(service_file = 'Budget Wrangling-e5de1c7718f7.json')

# Create empty dataframe

wkbook = gc.open('Budget')
#select the first sheet 
sheet = wkbook[1]

#Get the max Row number with the data to date. 
max_indx = len(sheet.get_col(1, include_tailing_empty = False))

#update the first sheet with df, starting at cell B2. 
max_indx

1340

In [375]:
df = compiled_data[:2]
sheet.set_dataframe(df,( max_indx + 1 ,1 ), copy_head =False)

1341

In [99]:





valid = {'Chase','Checking'}
    if file_name not in valid:
        raise ValueError("results: status must be one of %r." % valid)

IndentationError: unexpected indent (<ipython-input-99-25bc55409636>, line 2)